# Physical Object: Water Turbine

The `WaterTurbine` class models a hydropower turbine. It calculates the electrical power generated based on the flow rate of water passing through it and the hydraulic head difference across it. The model uses the standard hydropower equation:

**P = η * ρ * g * Q * H**

Where:
- P = Power (Watts)
- η = efficiency
- ρ = density of water
- g = acceleration of gravity
- Q = flow rate (outflow)
- H = head difference

### State Variables
- `outflow` (float): The flow of water passing through the turbine (m^3/s). This is equal to the inflow, up to the turbine's maximum capacity.
- `power` (float): The electrical power generated by the turbine (Watts).

### Parameters
- `efficiency` (float): The turbine's conversion efficiency (0.0 to 1.0).
- `max_flow_rate` (float): The maximum flow rate the turbine can handle (m^3/s).

## Simulation Example

The following simulation demonstrates the turbine's behavior under varying conditions. We will simulate a scenario where both the available inflow and the upstream head are changing over time, and we will observe the resulting power generation.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from swp.simulation_identification.physical_objects.water_turbine import WaterTurbine
from swp.core.interfaces import State, Parameters

# 1. Define initial state and parameters
initial_state = State(power=0, outflow=0)
parameters = Parameters(
    efficiency=0.85,
    max_flow_rate=150.0
)

# 2. Create a WaterTurbine instance
turbine = WaterTurbine(name="main_turbine", initial_state=initial_state, parameters=parameters)

# 3. Simulation settings
dt = 600 # 10-minute time steps
simulation_duration = 86400 # 1 day
num_steps = int(simulation_duration / dt)
history = []
inflows = []
upstream_heads = []

# 4. Run the simulation loop
downstream_head = 50.0 # Constant tailwater level
for t in range(num_steps):
    # Simulate a variable inflow (e.g. from a controlled gate)
    inflow = 75 + 75 * np.sin(2 * np.pi * t / num_steps)
    inflows.append(inflow)
    turbine.set_inflow(inflow)
    
    # Simulate a falling upstream head (e.g. reservoir level dropping)
    upstream_head = 100 - 10 * (t / num_steps)
    upstream_heads.append(upstream_head)
    
    action = {
        'upstream_head': upstream_head,
        'downstream_head': downstream_head
    }
    
    current_state = turbine.step(action=action, dt=dt)
    history.append(current_state.copy())

print("Turbine simulation complete.")

## Results and Visualization

The plots show how the generated power is a function of both the flow rate (`outflow`) and the head difference. Even when the outflow is at its peak, the power generation is lower towards the end of the simulation because the head has decreased.

In [ ]:
# Extract data from history
time_hours = [i * dt / 3600 for i in range(num_steps)]
outflows = [h['outflow'] for h in history]
power_mw = [h['power'] / 1e6 for h in history] # Convert to Megawatts

# Create a DataFrame
df = pd.DataFrame({
    'Time (hours)': time_hours,
    'Inflow (m^3/s)': inflows,
    'Outflow (m^3/s)': outflows,
    'Upstream Head (m)': upstream_heads,
    'Power (MW)': power_mw
})

print(df.head())

# Plot the results
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

# Flows and Head
ax1.plot(df['Time (hours)'], df['Outflow (m^3/s)'], label='Outflow')
ax1_twin = ax1.twinx()
ax1_twin.plot(df['Time (hours)'], df['Upstream Head (m)'], label='Upstream Head', color='gray', linestyle='--')
ax1.set_xlabel('Time (hours)')
ax1.set_ylabel('Outflow (m^3/s)')
ax1_twin.set_ylabel('Head (m)')
ax1.set_title('Turbine Performance')
ax1.grid(True)
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax1_twin.get_legend_handles_labels()
ax1_twin.legend(lines + lines2, labels + labels2, loc='upper right')

# Power
ax2.plot(df['Time (hours)'], df['Power (MW)'], label='Power Generation', color='green')
ax2.set_xlabel('Time (hours)')
ax2.set_ylabel('Power (MW)')
ax2.grid(True)
ax2.legend()

plt.tight_layout()
plt.show()